# Vegetation attributes and time-series extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the vegetation time-series from the MODIS dataset (i.e., LAI and NDVI).

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**
* Python>=3.6
* Jupyter
* geopandas=0.10.2
* glob
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**
* data/shapefiles/estreams_catchments.shp
* data/gee/vegetation/LAI/EStreams_modis_LAI_mean_gee_{}.csv. LAI time-series CSV-file(s) exported from GEE.
* data/gee/vegetation/NDVI/EStreams_modis_NDVI_mean_gee_{}.csv. NDVI time-series CSV-file(s) exported from GEE.

**Directory:**
* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Didan, K. MODIS/Terra Vegetation Indices 16-Day L3 Global 500m SIN Grid V061 [Data set]. ASA EOSDIS Land Processes Distributed Active Archive Center https://doi.org/10.5067/MODIS/MOD13A1.061 (2021).
* Myneni, R., Knyazikhin, Y. & Park, T. MODIS/Terra Leaf Area Index/FPAR 8-Day L4 Global 500m SIN Grid V061 [Data set]. NASA EOSDIS Land Processes Distributed Active Archive Center https://doi.org/10.5067/MODIS/MOD15A2H.061 (2021).

## License

* LAI and NDVI: Open access: "MODIS data and products acquired through the LP DAAC have no restrictions on subsequent use, sale, or redistribution." https://lpdaac.usgs.gov/products/mod13a1v061/; https://lpdaac.usgs.gov/products/mod15a2hv061/;  (Last access 23 November 2023) 

## Observations
* This notebook assumes that the GEE code to export LAI and NDVI mean time-series from the MODIS dataset (EStreams_landscape_timeseries_LAI_gee.txt; EStreams_landscape_timeseries_NDVI_gee.txt) were run before in the GEE platform and that the output CSV-files are locally available. 
* It is not possible to export the 15,000 catchments at one single CSV, so there might be many files with the time-series stored separetly. 

# Import modules

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
import glob

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/2530415399.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [3]:
# Non-editable variables:
PATH_OUTPUT_TS = "results/timeseries/vegetationindices"
PATH_OUTPUT_ST = "results/staticattributes"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [4]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,area_diff,area_calc,basin_id,geometry
0,HUGR020,"9,600",46.785,21.142,6444410,9011,HUGR020,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,6.536,9595.794,HUGR020,"POLYGON ((21.13208 46.77291, 21.13208 46.77375..."
1,HUGR021,"189,000",46.423,18.896,6442080,189538,HUGR021,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.284,188597.110,HUGR021,"POLYGON ((18.91708 46.41791, 18.91708 46.41625..."
2,HUGR022,"28,500",48.126,22.340,6444304,29057,HUGR022,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.917,28507.473,HUGR022,"POLYGON ((22.32875 48.10875, 22.32791 48.10875..."
3,HUGR023,"188,000",46.627,18.869,6442060,189092,HUGR023,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.577,188286.167,HUGR023,"POLYGON ((18.89041 46.62875, 18.88875 46.62708..."
4,HUGR025,"1,210",47.662,19.683,6444240,1222,HUGR025,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.982,1206.441,HUGR025,"POLYGON ((19.68124 47.66875, 19.68291 47.66875..."
5,HUGR026,110,46.891,20.498,6444420,26647,HUGR026,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-99.587,109.639,HUGR026,"POLYGON ((20.49958 46.93125, 20.49958 46.93125..."
6,HUGR027,"4,490",48.497,21.229,6444330,4515,HUGR027,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.554,4494.402,HUGR027,"POLYGON ((21.23458 48.49708, 21.23208 48.49708..."
7,HUGR028,"5,770",46.883,18.141,6442110,5884,HUGR028,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-1.937,5773.506,HUGR028,"POLYGON ((16.56458 46.93291, 16.56541 46.93291..."
8,HUGR029,"185,000",47.495,19.048,6442040,184893,HUGR029,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,0.058,184810.677,HUGR029,"POLYGON ((19.11291 47.48291, 19.11125 47.48458..."
9,HUGR030,"13,000",46.419,16.695,6446100,13033,HUGR030,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,-0.253,13044.665,HUGR030,"POLYGON ((16.63625 46.45625, 16.63541 46.45625..."


In [5]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

The total number of catchments to be processed are: 33


## GEE outputs
### Leaf Area index (LAI)

In [6]:
# Check the files in the subdirectory:
filenames = glob.glob("data/gee/vegetation/LAI/*.csv")
print("Number of files:", len(filenames))
print("First file:", filenames[0])

Number of files: 1
First file: data/gee/vegetation/LAI/EStreams_modis_LAI_mean_gee_0_49.csv


In [7]:
# First, we create an empty DataFrame for the data with a datetime index:
LAI_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))

# Loop for reading and concatenating the data:
for file in tqdm.tqdm(filenames):
    
    # Read the data from the CSV file:
    LAI_file = pd.read_csv(file)
    LAI_file.drop(["system:index", ".geo"], axis=1, inplace=True)
    LAI_file = LAI_file.T
    
    # Set columns based on the "basin_id" row and drop it
    LAI_file.columns = LAI_file.loc["basin_id", :].tolist()
    LAI_file.drop(["basin_id"], axis=0, inplace=True)
    
    # Convert the index to integers and sort it
    LAI_file.index = LAI_file.index.astype(int)
    LAI_file.sort_index(inplace=True)
    
    # Create a new DataFrame with datetime index and assign values
    LAI_file_df = pd.DataFrame(columns=LAI_file.columns)
    LAI_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
    LAI_file_df.loc[:, LAI_file.columns] = LAI_file
    LAI_file_df.set_index("dates", inplace=True)
    LAI_file_df.index.name = ""
    
    # Concatenate the DataFrames along the columns (axis=1)
    LAI_df = pd.concat([LAI_df, LAI_file_df], axis=1)
    
# Apply the scale factor from Google Earth Engine (GEE)
LAI_df = LAI_df * 0.01
LAI_df

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/3894163365.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  LAI_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))
  0%|          | 0/1 [00:00<?, ?it/s]

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/3894163365.py:22: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  LAI_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
100%|██████████| 1/1 [00:00<00:00, 58.19it/s]


,HUGR019,HUGR024,HUGR034,HUGR041,HUGR051,HUGR030,HUGR021,HUGR023,HUGR042,HUGR047,...,HUGR029,HUGR025,HUGR037,HUGR032,HUGR022,HUGR039,HUGR036,HUGR050,HUGR048,HUGR027
2001-01-31,0.031787,0.04669,0.031253,0.030771,0.030749,0.030887,0.040952,0.040978,0.028362,0.029089,...,0.041046,0.035747,0.038313,0.041072,0.029324,0.038104,0.039498,0.03096,0.029395,0.059422
2001-02-28,0.029289,0.033464,0.050253,0.051022,0.051424,0.051338,0.037447,0.03746,0.026216,0.023813,...,0.0376,0.026804,0.049163,0.037661,0.026703,0.031731,0.031323,0.036049,0.025801,0.035992
2001-03-31,0.055038,0.052918,0.090397,0.090213,0.089371,0.087401,0.068,0.067998,0.060734,0.056415,...,0.068027,0.047994,0.09251,0.068136,0.055156,0.053182,0.055021,0.063282,0.053998,0.054247
2001-04-30,0.098474,0.096276,0.12721,0.12316,0.119168,0.118595,0.103265,0.103236,0.132973,0.118214,...,0.102819,0.117489,0.136846,0.10281,0.098839,0.106919,0.113475,0.106701,0.098544,0.085868
2001-05-31,0.297891,0.365267,0.237209,0.232772,0.229168,0.23596,0.258126,0.25825,0.327665,0.285231,...,0.258903,0.317954,0.265758,0.258836,0.2873,0.344883,0.331384,0.350565,0.277707,0.31279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.26314,0.315962,0.247268,0.250999,0.2533,0.25898,0.221677,0.221893,0.250974,0.226486,...,0.223902,0.213199,0.188265,0.224125,0.276546,0.292275,0.272211,0.292005,0.261186,0.287825
2022-09-30,0.157072,0.244625,0.20977,0.211419,0.21208,0.207916,0.174291,0.174428,0.189423,0.168175,...,0.175654,0.196759,0.151001,0.175719,0.152509,0.244405,0.228267,0.147903,0.148901,0.191104
2022-10-31,0.10668,0.131088,0.104659,0.10931,0.114238,0.108904,0.110184,0.110265,0.137554,0.12165,...,0.111239,0.112301,0.087653,0.111351,0.117828,0.13951,0.128572,0.092922,0.116199,0.115626
2022-11-30,0.063163,0.076506,0.075035,0.076209,0.077631,0.079397,0.093833,0.0939,0.067574,0.073382,...,0.094747,0.049291,0.069457,0.094974,0.056889,0.067955,0.06798,0.082662,0.057619,0.073685


In [8]:
# Here we add the columns of the catchemnts that were not processed
# Adding new columns with NaN values only if they don't exist
for col in catchment_boundaries.basin_id.tolist():
    if col not in LAI_df.columns:
        LAI_df[col] = np.nan
LAI_df

,HUGR019,HUGR024,HUGR034,HUGR041,HUGR051,HUGR030,HUGR021,HUGR023,HUGR042,HUGR047,...,HUGR029,HUGR025,HUGR037,HUGR032,HUGR022,HUGR039,HUGR036,HUGR050,HUGR048,HUGR027
2001-01-31,0.031787,0.04669,0.031253,0.030771,0.030749,0.030887,0.040952,0.040978,0.028362,0.029089,...,0.041046,0.035747,0.038313,0.041072,0.029324,0.038104,0.039498,0.03096,0.029395,0.059422
2001-02-28,0.029289,0.033464,0.050253,0.051022,0.051424,0.051338,0.037447,0.03746,0.026216,0.023813,...,0.0376,0.026804,0.049163,0.037661,0.026703,0.031731,0.031323,0.036049,0.025801,0.035992
2001-03-31,0.055038,0.052918,0.090397,0.090213,0.089371,0.087401,0.068,0.067998,0.060734,0.056415,...,0.068027,0.047994,0.09251,0.068136,0.055156,0.053182,0.055021,0.063282,0.053998,0.054247
2001-04-30,0.098474,0.096276,0.12721,0.12316,0.119168,0.118595,0.103265,0.103236,0.132973,0.118214,...,0.102819,0.117489,0.136846,0.10281,0.098839,0.106919,0.113475,0.106701,0.098544,0.085868
2001-05-31,0.297891,0.365267,0.237209,0.232772,0.229168,0.23596,0.258126,0.25825,0.327665,0.285231,...,0.258903,0.317954,0.265758,0.258836,0.2873,0.344883,0.331384,0.350565,0.277707,0.31279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.26314,0.315962,0.247268,0.250999,0.2533,0.25898,0.221677,0.221893,0.250974,0.226486,...,0.223902,0.213199,0.188265,0.224125,0.276546,0.292275,0.272211,0.292005,0.261186,0.287825
2022-09-30,0.157072,0.244625,0.20977,0.211419,0.21208,0.207916,0.174291,0.174428,0.189423,0.168175,...,0.175654,0.196759,0.151001,0.175719,0.152509,0.244405,0.228267,0.147903,0.148901,0.191104
2022-10-31,0.10668,0.131088,0.104659,0.10931,0.114238,0.108904,0.110184,0.110265,0.137554,0.12165,...,0.111239,0.112301,0.087653,0.111351,0.117828,0.13951,0.128572,0.092922,0.116199,0.115626
2022-11-30,0.063163,0.076506,0.075035,0.076209,0.077631,0.079397,0.093833,0.0939,0.067574,0.073382,...,0.094747,0.049291,0.069457,0.094974,0.056889,0.067955,0.06798,0.082662,0.057619,0.073685


In [9]:
# Here we sort the columns:
LAI_df = LAI_df.sort_index(axis=1)
LAI_df

,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
2001-01-31,0.031787,0.02824,0.040952,0.029324,0.040978,0.04669,0.035747,0.023184,0.059422,0.034193,...,0.028362,0.027744,0.03197,0.039982,0.029973,0.029089,0.029395,0.02107,0.03096,0.030749
2001-02-28,0.029289,0.023929,0.037447,0.026703,0.03746,0.033464,0.026804,0.01603,0.035992,0.0431,...,0.026216,0.01785,0.029052,0.027695,0.023215,0.023813,0.025801,0.016795,0.036049,0.051424
2001-03-31,0.055038,0.057499,0.068,0.055156,0.067998,0.052918,0.047994,0.073792,0.054247,0.082975,...,0.060734,0.050445,0.054991,0.062449,0.056804,0.056415,0.053998,0.054238,0.063282,0.089371
2001-04-30,0.098474,0.127526,0.103265,0.098839,0.103236,0.096276,0.117489,0.251032,0.085868,0.136328,...,0.132973,0.110526,0.100257,0.106498,0.129312,0.118214,0.098544,0.171311,0.106701,0.119168
2001-05-31,0.297891,0.299721,0.258126,0.2873,0.25825,0.365267,0.317954,0.300635,0.31279,0.272974,...,0.327665,0.221992,0.29685,0.263096,0.285163,0.285231,0.277707,0.267718,0.350565,0.229168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.26314,0.223381,0.221677,0.276546,0.221893,0.315962,0.213199,0.043504,0.287825,0.196786,...,0.250974,0.133657,0.255591,0.219156,0.195126,0.226486,0.261186,0.058923,0.292005,0.2533
2022-09-30,0.157072,0.169436,0.174291,0.152509,0.174428,0.244625,0.196759,0.03865,0.191104,0.180284,...,0.189423,0.115226,0.155116,0.159224,0.151551,0.168175,0.148901,0.095914,0.147903,0.21208
2022-10-31,0.10668,0.123691,0.110184,0.117828,0.110265,0.131088,0.112301,0.028533,0.115626,0.080074,...,0.137554,0.107858,0.105837,0.129485,0.114806,0.12165,0.116199,0.087922,0.092922,0.114238
2022-11-30,0.063163,0.068599,0.093833,0.056889,0.0939,0.076506,0.049291,0.037798,0.073685,0.067871,...,0.067574,0.074866,0.06266,0.078495,0.066626,0.073382,0.057619,0.055566,0.082662,0.077631


In [10]:
# Resample to yearly mean
LAI_yr = LAI_df.resample('Y').mean()
LAI_yr

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/3545936846.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  LAI_yr = LAI_df.resample('Y').mean()


,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
2001-12-31,0.148977,0.145569,0.133691,0.153454,0.13375,0.162289,0.146657,0.098259,0.146238,0.145201,...,0.157853,0.120011,0.147145,0.142873,0.137054,0.142039,0.149159,0.104326,0.161218,0.161254
2002-12-31,0.149101,0.14338,0.147292,0.149029,0.147386,0.181319,0.143895,0.078275,0.160665,0.137601,...,0.156223,0.116462,0.147047,0.14427,0.133656,0.141507,0.144507,0.092814,0.162261,0.159153
2003-12-31,0.140169,0.136748,0.134447,0.136331,0.134543,0.176584,0.142651,0.058908,0.159673,0.134404,...,0.150989,0.101466,0.138281,0.131139,0.124982,0.134589,0.131581,0.076878,0.156068,0.150444
2004-12-31,0.148531,0.14774,0.135863,0.145992,0.135925,0.174321,0.154771,0.110785,0.160454,0.141829,...,0.156882,0.12239,0.147271,0.145959,0.139647,0.147488,0.142392,0.106582,0.160748,0.151101
2005-12-31,0.159421,0.153837,0.142159,0.159266,0.142228,0.191083,0.155126,0.075671,0.167188,0.149084,...,0.165785,0.125611,0.157661,0.155085,0.143374,0.154313,0.154663,0.09994,0.174508,0.152974
2006-12-31,0.157063,0.154188,0.139649,0.158099,0.139717,0.180026,0.155247,0.079285,0.165639,0.146317,...,0.166703,0.120697,0.154746,0.151453,0.141658,0.153882,0.153346,0.091019,0.173067,0.152198
2007-12-31,0.161557,0.156083,0.150825,0.164218,0.150932,0.193311,0.152383,0.082337,0.170624,0.150196,...,0.169793,0.116608,0.159164,0.14826,0.142539,0.15499,0.158714,0.092025,0.174679,0.161606
2008-12-31,0.166721,0.158476,0.146227,0.168731,0.146304,0.183374,0.162324,0.101905,0.167265,0.15025,...,0.168758,0.135485,0.16494,0.154954,0.148271,0.159053,0.164719,0.10994,0.18068,0.150376
2009-12-31,0.164807,0.150097,0.144608,0.166961,0.144708,0.193108,0.15719,0.073599,0.170275,0.153854,...,0.165919,0.116956,0.162159,0.146627,0.137815,0.147253,0.161133,0.085471,0.182582,0.155351
2010-12-31,0.155335,0.156393,0.132048,0.158819,0.132118,0.176008,0.152575,0.085323,0.158371,0.146877,...,0.168756,0.129188,0.153465,0.146813,0.145014,0.154827,0.154802,0.111629,0.165929,0.146999


In [11]:
# Calculate the mean for each month across all years (monht of the year)
LAI_moy = LAI_df.groupby(LAI_df.index.month).mean()

# Rename the index to the three-letter month abbreviations
LAI_moy.index = pd.to_datetime(LAI_moy.index, format='%m').strftime('%b')

LAI_moy

,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
Jan,0.038117,0.028106,0.050049,0.031279,0.050089,0.054877,0.043016,0.027859,0.065174,0.032908,...,0.028152,0.035786,0.038092,0.033173,0.028988,0.028918,0.031623,0.032194,0.039539,0.045532
Feb,0.025677,0.029752,0.035054,0.025473,0.035071,0.0307,0.027334,0.020992,0.030465,0.033086,...,0.031284,0.024418,0.025636,0.030561,0.028394,0.03,0.024974,0.024123,0.025985,0.040278
Mar,0.050733,0.062158,0.066027,0.050535,0.066043,0.057172,0.055415,0.06229,0.054534,0.064145,...,0.064948,0.05427,0.050999,0.057768,0.060026,0.060444,0.05017,0.062889,0.052744,0.069444
Apr,0.123411,0.151405,0.13901,0.120211,0.139056,0.137258,0.149975,0.161611,0.11257,0.149769,...,0.160023,0.125591,0.12478,0.126398,0.14393,0.146577,0.119129,0.155255,0.138878,0.12909
May,0.280378,0.2892,0.237007,0.274572,0.237137,0.331201,0.299034,0.207253,0.282612,0.268827,...,0.31929,0.215852,0.278956,0.25503,0.266445,0.278837,0.266457,0.214102,0.326537,0.216144
Jun,0.330552,0.304872,0.283835,0.335561,0.283981,0.375213,0.32571,0.159381,0.347336,0.294452,...,0.333875,0.242267,0.325765,0.30179,0.282788,0.297273,0.32463,0.182855,0.356832,0.283945
Jul,0.328088,0.301601,0.264429,0.339178,0.264535,0.361273,0.281344,0.127151,0.33158,0.287196,...,0.329372,0.242786,0.321584,0.299085,0.276723,0.296699,0.32867,0.153272,0.345725,0.316251
Aug,0.310455,0.282285,0.242227,0.318212,0.242365,0.36129,0.28118,0.091786,0.321961,0.262785,...,0.313152,0.212829,0.304004,0.279537,0.254455,0.280028,0.306307,0.122437,0.339169,0.296548
Sep,0.226734,0.207199,0.200357,0.226164,0.200537,0.286324,0.220417,0.046629,0.246624,0.202934,...,0.23051,0.140289,0.222011,0.208101,0.184904,0.208878,0.216172,0.083647,0.260438,0.235904
Oct,0.103129,0.109594,0.107345,0.109285,0.107437,0.118461,0.109848,0.026727,0.102236,0.111363,...,0.118574,0.078355,0.101573,0.106681,0.098473,0.112761,0.105338,0.056614,0.107763,0.123985


### Normalized Vegetation Difference Index (NDVI)

In [12]:
# Check the files in the subdirectory:
filenames = glob.glob("data/gee/vegetation/NDVI/*.csv")
print("Number of files:", len(filenames))
print("First file:", filenames[0])

Number of files: 1
First file: data/gee/vegetation/NDVI/EStreams_modis_NDVI_mean_gee_0_49.csv


In [13]:
# First, we create an empty DataFrame for the data with a datetime index:
ndvi_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))

# Loop for reading and concatenating the data:
for file in tqdm.tqdm(filenames):
    
    # Read the data from the CSV file:
    ndvi_file = pd.read_csv(file)
    ndvi_file.drop(["system:index", ".geo"], axis=1, inplace=True)
    ndvi_file = ndvi_file.T
    
    # Set columns based on the "basin_id" row and drop it
    ndvi_file.columns = ndvi_file.loc["basin_id", :].tolist()
    ndvi_file.drop(["basin_id"], axis=0, inplace=True)
    
    # Convert the index to integers and sort it
    ndvi_file.index = ndvi_file.index.astype(int)
    ndvi_file.sort_index(inplace=True)
    
    # Create a new DataFrame with datetime index and assign values
    ndvi_file_df = pd.DataFrame(columns=ndvi_file.columns)
    ndvi_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
    ndvi_file_df.loc[:, ndvi_file.columns] = ndvi_file
    ndvi_file_df.set_index("dates", inplace=True)
    ndvi_file_df.index.name = ""
    
    # Concatenate the DataFrames along the columns (axis=1)
    ndvi_df = pd.concat([ndvi_df, ndvi_file_df], axis=1)
    
# Apply the scale factor from Google Earth Engine (GEE)
ndvi_df = ndvi_df * 0.0001
ndvi_df

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/734810479.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ndvi_df = pd.DataFrame(index=pd.date_range(start='2001-01-01', end='2022-12-31', freq='M'))
  0%|          | 0/1 [00:00<?, ?it/s]/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/734810479.py:22: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ndvi_file_df["dates"] = pd.date_range(start='2001-01-01', end='2022-12-31', freq='M')
100%|██████████| 1/1 [00:00<00:00, 69.86it/s]


,HUGR019,HUGR024,HUGR034,HUGR041,HUGR051,HUGR030,HUGR021,HUGR023,HUGR042,HUGR047,...,HUGR029,HUGR025,HUGR037,HUGR032,HUGR022,HUGR039,HUGR036,HUGR050,HUGR048,HUGR027
2001-01-31,0.334911,0.431899,0.29587,0.282143,0.273255,0.284354,0.289825,0.289664,0.378183,0.371938,...,0.287912,0.375309,0.407812,0.287595,0.29005,0.406107,0.40841,0.37499,0.295956,0.346667
2001-02-28,0.289739,0.346346,0.398491,0.39646,0.392051,0.390627,0.338212,0.33818,0.267572,0.248479,...,0.338122,0.342953,0.436875,0.338367,0.245943,0.344738,0.346082,0.338739,0.248277,0.330445
2001-03-31,0.425078,0.434938,0.489618,0.486086,0.478813,0.496805,0.460685,0.46069,0.446327,0.448448,...,0.460683,0.406816,0.521345,0.461253,0.426895,0.420869,0.423811,0.441291,0.423622,0.439209
2001-04-30,0.563878,0.579296,0.536215,0.524981,0.513595,0.532867,0.521636,0.521596,0.631153,0.612983,...,0.520563,0.594277,0.609894,0.520437,0.561298,0.597332,0.610719,0.58453,0.559567,0.543774
2001-05-31,0.744431,0.803674,0.666656,0.663347,0.66107,0.68689,0.694841,0.694991,0.75907,0.733695,...,0.695969,0.768766,0.690953,0.696187,0.742521,0.778429,0.764337,0.77552,0.73363,0.769682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.697299,0.744864,0.713002,0.718637,0.722596,0.733796,0.667477,0.667816,0.667712,0.670093,...,0.670894,0.655447,0.619318,0.671382,0.732658,0.730169,0.70485,0.691061,0.714913,0.714379
2022-09-30,0.644392,0.721197,0.682443,0.685358,0.68656,0.698127,0.647833,0.648045,0.687993,0.693782,...,0.649883,0.687923,0.613435,0.650065,0.621596,0.724082,0.69649,0.615483,0.631638,0.700529
2022-10-31,0.404281,0.511197,0.579985,0.590448,0.599724,0.625049,0.587687,0.587893,0.579276,0.557745,...,0.59497,0.458896,0.564472,0.597416,0.500424,0.517245,0.499788,0.224743,0.504267,0.551502
2022-11-30,0.5399,0.541453,0.559682,0.558344,0.555599,0.583201,0.551856,0.551939,0.561892,0.558244,...,0.552542,0.549415,0.567746,0.552734,0.547683,0.553893,0.542381,0.540371,0.549609,0.522772


In [14]:
# Here we add the columns of the catchemnts that were not processed
# Adding new columns with NaN values only if they don't exist
for col in catchment_boundaries.basin_id.tolist():
    if col not in ndvi_df.columns:
        ndvi_df[col] = np.nan
ndvi_df

,HUGR019,HUGR024,HUGR034,HUGR041,HUGR051,HUGR030,HUGR021,HUGR023,HUGR042,HUGR047,...,HUGR029,HUGR025,HUGR037,HUGR032,HUGR022,HUGR039,HUGR036,HUGR050,HUGR048,HUGR027
2001-01-31,0.334911,0.431899,0.29587,0.282143,0.273255,0.284354,0.289825,0.289664,0.378183,0.371938,...,0.287912,0.375309,0.407812,0.287595,0.29005,0.406107,0.40841,0.37499,0.295956,0.346667
2001-02-28,0.289739,0.346346,0.398491,0.39646,0.392051,0.390627,0.338212,0.33818,0.267572,0.248479,...,0.338122,0.342953,0.436875,0.338367,0.245943,0.344738,0.346082,0.338739,0.248277,0.330445
2001-03-31,0.425078,0.434938,0.489618,0.486086,0.478813,0.496805,0.460685,0.46069,0.446327,0.448448,...,0.460683,0.406816,0.521345,0.461253,0.426895,0.420869,0.423811,0.441291,0.423622,0.439209
2001-04-30,0.563878,0.579296,0.536215,0.524981,0.513595,0.532867,0.521636,0.521596,0.631153,0.612983,...,0.520563,0.594277,0.609894,0.520437,0.561298,0.597332,0.610719,0.58453,0.559567,0.543774
2001-05-31,0.744431,0.803674,0.666656,0.663347,0.66107,0.68689,0.694841,0.694991,0.75907,0.733695,...,0.695969,0.768766,0.690953,0.696187,0.742521,0.778429,0.764337,0.77552,0.73363,0.769682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.697299,0.744864,0.713002,0.718637,0.722596,0.733796,0.667477,0.667816,0.667712,0.670093,...,0.670894,0.655447,0.619318,0.671382,0.732658,0.730169,0.70485,0.691061,0.714913,0.714379
2022-09-30,0.644392,0.721197,0.682443,0.685358,0.68656,0.698127,0.647833,0.648045,0.687993,0.693782,...,0.649883,0.687923,0.613435,0.650065,0.621596,0.724082,0.69649,0.615483,0.631638,0.700529
2022-10-31,0.404281,0.511197,0.579985,0.590448,0.599724,0.625049,0.587687,0.587893,0.579276,0.557745,...,0.59497,0.458896,0.564472,0.597416,0.500424,0.517245,0.499788,0.224743,0.504267,0.551502
2022-11-30,0.5399,0.541453,0.559682,0.558344,0.555599,0.583201,0.551856,0.551939,0.561892,0.558244,...,0.552542,0.549415,0.567746,0.552734,0.547683,0.553893,0.542381,0.540371,0.549609,0.522772


In [15]:
# Here we sort the columns:
ndvi_df = ndvi_df.sort_index(axis=1)
ndvi_df

,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
2001-01-31,0.334911,0.36691,0.289825,0.29005,0.289664,0.431899,0.375309,0.373014,0.346667,0.377835,...,0.378183,0.345534,0.336639,0.372439,0.356084,0.371938,0.295956,0.316598,0.37499,0.273255
2001-02-28,0.289739,0.257267,0.338212,0.245943,0.33818,0.346346,0.342953,0.336558,0.330445,0.394104,...,0.267572,0.264683,0.292149,0.279094,0.2655,0.248479,0.248277,0.307189,0.338739,0.392051
2001-03-31,0.425078,0.444702,0.460685,0.426895,0.46069,0.434938,0.406816,0.547982,0.439209,0.489871,...,0.446327,0.419311,0.425757,0.452308,0.447166,0.448448,0.423622,0.460262,0.441291,0.478813
2001-04-30,0.563878,0.62442,0.521636,0.561298,0.521596,0.579296,0.594277,0.748655,0.543774,0.589826,...,0.631153,0.562707,0.567728,0.571864,0.622923,0.612983,0.559567,0.679935,0.58453,0.513595
2001-05-31,0.744431,0.741039,0.694841,0.742521,0.694991,0.803674,0.768766,0.737896,0.769682,0.663879,...,0.75907,0.668751,0.743655,0.719225,0.732359,0.733695,0.73363,0.717054,0.77552,0.66107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,0.697299,0.646037,0.667477,0.732658,0.667816,0.744864,0.655447,0.322229,0.714379,0.615423,...,0.667712,0.549243,0.686125,0.671223,0.613058,0.670093,0.714913,0.418382,0.691061,0.722596
2022-09-30,0.644392,0.68309,0.647833,0.621596,0.648045,0.721197,0.687923,0.426348,0.700529,0.621037,...,0.687993,0.682069,0.64331,0.690649,0.66588,0.693782,0.631638,0.609849,0.615483,0.68656
2022-10-31,0.404281,0.557647,0.587687,0.500424,0.587893,0.511197,0.458896,0.541262,0.551502,0.50844,...,0.579276,0.581662,0.407391,0.607778,0.558725,0.557745,0.504267,0.559602,0.224743,0.599724
2022-11-30,0.5399,0.556569,0.551856,0.547683,0.551939,0.541453,0.549415,0.395181,0.522772,0.526949,...,0.561892,0.587142,0.540232,0.577287,0.550905,0.558244,0.549609,0.53786,0.540371,0.555599


In [16]:
# Resample to yearly mean
ndvi_yr = ndvi_df.resample('Y').mean()
ndvi_yr

/var/folders/_g/v_sd2vds19n_f5ckymsg1h4r0000gn/T/ipykernel_77966/2251271537.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  ndvi_yr = ndvi_df.resample('Y').mean()


,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
2001-12-31,0.547209,0.550862,0.519984,0.551047,0.520065,0.563629,0.523504,0.457566,0.547099,0.539381,...,0.562356,0.51231,0.544176,0.551136,0.539477,0.554367,0.546952,0.502496,0.564156,0.565528
2002-12-31,0.538191,0.55347,0.547277,0.536133,0.547417,0.57958,0.549992,0.44939,0.566088,0.524023,...,0.55937,0.526923,0.535736,0.56349,0.543711,0.562759,0.534348,0.500544,0.542102,0.567517
2003-12-31,0.490634,0.50285,0.495163,0.48753,0.495318,0.534693,0.501528,0.365021,0.525453,0.476892,...,0.516418,0.451069,0.488316,0.499955,0.486728,0.50773,0.482771,0.41641,0.501777,0.529364
2004-12-31,0.518195,0.543398,0.510581,0.509813,0.510627,0.559523,0.547903,0.508152,0.541247,0.518336,...,0.545314,0.530581,0.518044,0.542832,0.538896,0.547161,0.511451,0.526768,0.514921,0.520518
2005-12-31,0.528939,0.548149,0.503172,0.521889,0.503232,0.584127,0.556927,0.428825,0.550235,0.522324,...,0.556842,0.510093,0.528021,0.541574,0.53426,0.55747,0.520489,0.49265,0.532996,0.532643
2006-12-31,0.52406,0.54455,0.495397,0.529684,0.495435,0.545384,0.523529,0.433456,0.530984,0.501917,...,0.55557,0.499052,0.521231,0.540607,0.530022,0.548645,0.525462,0.47657,0.529732,0.517902
2007-12-31,0.570435,0.562703,0.551132,0.574286,0.551288,0.612997,0.554807,0.460159,0.586815,0.543746,...,0.575791,0.527715,0.56715,0.556017,0.548427,0.564628,0.569837,0.497547,0.576489,0.572289
2008-12-31,0.5679,0.57113,0.539912,0.569234,0.540041,0.588532,0.564339,0.457836,0.572105,0.525681,...,0.577616,0.545274,0.565605,0.56458,0.558037,0.578727,0.567258,0.510122,0.580106,0.541802
2009-12-31,0.541024,0.526962,0.522908,0.547383,0.523048,0.583006,0.539444,0.382425,0.549679,0.532693,...,0.545202,0.477049,0.537176,0.522598,0.509004,0.52559,0.539122,0.425157,0.552276,0.536003
2010-12-31,0.536859,0.549766,0.476876,0.546712,0.476946,0.551248,0.53834,0.4275,0.535776,0.518504,...,0.563873,0.498481,0.534926,0.5415,0.536406,0.55162,0.541922,0.493084,0.542585,0.511655


In [17]:
# Calculate the mean for each month across all years (monht of the year)
ndvi_moy = ndvi_df.groupby(ndvi_df.index.month).mean()

# Rename the index to the three-letter month abbreviations
ndvi_moy.index = pd.to_datetime(ndvi_moy.index, format='%m').strftime('%b')

ndvi_moy

,HUGR019,HUGR020,HUGR021,HUGR022,HUGR023,HUGR024,HUGR025,HUGR026,HUGR027,HUGR028,...,HUGR042,HUGR043,HUGR044,HUGR045,HUGR046,HUGR047,HUGR048,HUGR049,HUGR050,HUGR051
Jan,0.243804,0.320859,0.283509,0.219328,0.283451,0.294022,0.327435,0.300705,0.26466,0.316475,...,0.328692,0.306646,0.24646,0.292059,0.312703,0.314507,0.227007,0.324169,0.236052,0.310915
Feb,0.260978,0.320837,0.28913,0.242901,0.289089,0.296537,0.309832,0.300326,0.27161,0.312419,...,0.325593,0.30516,0.263077,0.300755,0.313987,0.319026,0.247627,0.312624,0.259862,0.285831
Mar,0.378259,0.44009,0.409302,0.368259,0.409323,0.408913,0.41858,0.44656,0.393793,0.414058,...,0.446595,0.410022,0.380227,0.406797,0.431825,0.432925,0.370868,0.444545,0.38609,0.393468
Apr,0.577835,0.618848,0.57042,0.574766,0.570501,0.608344,0.618494,0.617455,0.56994,0.588768,...,0.631184,0.571542,0.579764,0.583723,0.608641,0.612627,0.57193,0.623969,0.601731,0.538935
May,0.746498,0.745628,0.692899,0.749237,0.693038,0.790177,0.770815,0.666105,0.759909,0.69014,...,0.767859,0.681669,0.745083,0.728726,0.729337,0.740577,0.740883,0.681702,0.775238,0.657783
Jun,0.775497,0.741683,0.721949,0.788706,0.722047,0.800123,0.742953,0.566398,0.785291,0.70081,...,0.759737,0.701478,0.769844,0.751888,0.725173,0.741945,0.77931,0.626849,0.78387,0.732441
Jul,0.767648,0.728822,0.696273,0.785428,0.69636,0.789358,0.702746,0.520219,0.771139,0.689948,...,0.74451,0.693772,0.760234,0.739453,0.708275,0.735949,0.776393,0.58007,0.768571,0.757106
Aug,0.745709,0.698778,0.686636,0.764863,0.686818,0.778588,0.69798,0.445822,0.760384,0.674858,...,0.716646,0.65043,0.738182,0.718065,0.675236,0.712775,0.753393,0.53484,0.752152,0.746497
Sep,0.685737,0.650362,0.660782,0.700895,0.661056,0.726809,0.660526,0.382823,0.713204,0.636398,...,0.665971,0.58682,0.679229,0.671967,0.626036,0.669921,0.689713,0.500968,0.695579,0.704958
Oct,0.576979,0.581321,0.589438,0.598655,0.589674,0.595877,0.567388,0.382461,0.600826,0.578626,...,0.592648,0.526226,0.573125,0.595187,0.562125,0.597075,0.590576,0.479113,0.570111,0.616828


# Final aggregation (static attributes)

In [18]:
# LAI:
LAI_moy_T = LAI_moy.T
LAI_moy_T.columns = pd.to_datetime(LAI_moy_T.columns, format='%b').strftime('%m')
LAI_moy_T.columns = "lai_" + LAI_moy_T.columns

LAI_moy_T["lai_mean"] = LAI_moy_T.mean(axis = 1)

LAI_moy_T

,lai_01,lai_02,lai_03,lai_04,lai_05,lai_06,lai_07,lai_08,lai_09,lai_10,lai_11,lai_12,lai_mean
HUGR019,0.038117,0.025677,0.050733,0.123411,0.280378,0.330552,0.328088,0.310455,0.226734,0.103129,0.05554,0.076808,0.162469
HUGR020,0.028106,0.029752,0.062158,0.151405,0.2892,0.304872,0.301601,0.282285,0.207199,0.109594,0.054437,0.053728,0.156195
HUGR021,0.050049,0.035054,0.066027,0.13901,0.237007,0.283835,0.264429,0.242227,0.200357,0.107345,0.06993,0.10858,0.150321
HUGR022,0.031279,0.025473,0.050535,0.120211,0.274572,0.335561,0.339178,0.318212,0.226164,0.109285,0.057116,0.072028,0.163301
HUGR023,0.050089,0.035071,0.066043,0.139056,0.237137,0.283981,0.264535,0.242365,0.200537,0.107437,0.069987,0.108675,0.150409
HUGR024,0.054877,0.0307,0.057172,0.137258,0.331201,0.375213,0.361273,0.36129,0.286324,0.118461,0.058161,0.095865,0.188983
HUGR025,0.043016,0.027334,0.055415,0.149975,0.299034,0.32571,0.281344,0.28118,0.220417,0.109848,0.047636,0.092056,0.16108
HUGR026,0.027859,0.020992,0.06229,0.161611,0.207253,0.159381,0.127151,0.091786,0.046629,0.026727,0.024964,0.068477,0.085427
HUGR027,0.065174,0.030465,0.054534,0.11257,0.282612,0.347336,0.33158,0.321961,0.246624,0.102236,0.067259,0.107378,0.172477
HUGR028,0.032908,0.033086,0.064145,0.149769,0.268827,0.294452,0.287196,0.262785,0.202934,0.111363,0.05571,0.065497,0.152389


In [19]:
# NDVI:
ndvi_moy_T = ndvi_moy.T
ndvi_moy_T.columns = pd.to_datetime(ndvi_moy_T.columns, format='%b').strftime('%m')
ndvi_moy_T.columns = "ndvi_" + ndvi_moy_T.columns

ndvi_moy_T["ndvi_mean"] = ndvi_moy_T.mean(axis = 1)

ndvi_moy_T

,ndvi_01,ndvi_02,ndvi_03,ndvi_04,ndvi_05,ndvi_06,ndvi_07,ndvi_08,ndvi_09,ndvi_10,ndvi_11,ndvi_12,ndvi_mean
HUGR019,0.243804,0.260978,0.378259,0.577835,0.746498,0.775497,0.767648,0.745709,0.685737,0.576979,0.486502,0.337797,0.548604
HUGR020,0.320859,0.320837,0.44009,0.618848,0.745628,0.741683,0.728822,0.698778,0.650362,0.581321,0.495604,0.391456,0.561191
HUGR021,0.283509,0.28913,0.409302,0.57042,0.692899,0.721949,0.696273,0.686636,0.660782,0.589438,0.498197,0.379523,0.539838
HUGR022,0.219328,0.242901,0.368259,0.574766,0.749237,0.788706,0.785428,0.764863,0.700895,0.598655,0.495497,0.321582,0.550843
HUGR023,0.283451,0.289089,0.409323,0.570501,0.693038,0.722047,0.69636,0.686818,0.661056,0.589674,0.498324,0.379554,0.539936
HUGR024,0.294022,0.296537,0.408913,0.608344,0.790177,0.800123,0.789358,0.778588,0.726809,0.595877,0.498424,0.382712,0.580823
HUGR025,0.327435,0.309832,0.41858,0.618494,0.770815,0.742953,0.702746,0.69798,0.660526,0.567388,0.478336,0.397457,0.557712
HUGR026,0.300705,0.300326,0.44656,0.617455,0.666105,0.566398,0.520219,0.445822,0.382823,0.382461,0.393705,0.35572,0.448192
HUGR027,0.26466,0.27161,0.393793,0.56994,0.759909,0.785291,0.771139,0.760384,0.713204,0.600826,0.502033,0.346595,0.561615
HUGR028,0.316475,0.312419,0.414058,0.588768,0.69014,0.70081,0.689948,0.674858,0.636398,0.578626,0.486094,0.385912,0.539542


In [20]:
# First we create an empty table data frame to assing the values to it
vegetation_df = pd.DataFrame(index = ndvi_moy_T.index)

# Now we proceed with the concatenation:
vegetation_df = pd.concat([LAI_moy_T, ndvi_moy_T], axis=1)

vegetation_df

,lai_01,lai_02,lai_03,lai_04,lai_05,lai_06,lai_07,lai_08,lai_09,lai_10,...,ndvi_04,ndvi_05,ndvi_06,ndvi_07,ndvi_08,ndvi_09,ndvi_10,ndvi_11,ndvi_12,ndvi_mean
HUGR019,0.038117,0.025677,0.050733,0.123411,0.280378,0.330552,0.328088,0.310455,0.226734,0.103129,...,0.577835,0.746498,0.775497,0.767648,0.745709,0.685737,0.576979,0.486502,0.337797,0.548604
HUGR020,0.028106,0.029752,0.062158,0.151405,0.2892,0.304872,0.301601,0.282285,0.207199,0.109594,...,0.618848,0.745628,0.741683,0.728822,0.698778,0.650362,0.581321,0.495604,0.391456,0.561191
HUGR021,0.050049,0.035054,0.066027,0.13901,0.237007,0.283835,0.264429,0.242227,0.200357,0.107345,...,0.57042,0.692899,0.721949,0.696273,0.686636,0.660782,0.589438,0.498197,0.379523,0.539838
HUGR022,0.031279,0.025473,0.050535,0.120211,0.274572,0.335561,0.339178,0.318212,0.226164,0.109285,...,0.574766,0.749237,0.788706,0.785428,0.764863,0.700895,0.598655,0.495497,0.321582,0.550843
HUGR023,0.050089,0.035071,0.066043,0.139056,0.237137,0.283981,0.264535,0.242365,0.200537,0.107437,...,0.570501,0.693038,0.722047,0.69636,0.686818,0.661056,0.589674,0.498324,0.379554,0.539936
HUGR024,0.054877,0.0307,0.057172,0.137258,0.331201,0.375213,0.361273,0.36129,0.286324,0.118461,...,0.608344,0.790177,0.800123,0.789358,0.778588,0.726809,0.595877,0.498424,0.382712,0.580823
HUGR025,0.043016,0.027334,0.055415,0.149975,0.299034,0.32571,0.281344,0.28118,0.220417,0.109848,...,0.618494,0.770815,0.742953,0.702746,0.69798,0.660526,0.567388,0.478336,0.397457,0.557712
HUGR026,0.027859,0.020992,0.06229,0.161611,0.207253,0.159381,0.127151,0.091786,0.046629,0.026727,...,0.617455,0.666105,0.566398,0.520219,0.445822,0.382823,0.382461,0.393705,0.35572,0.448192
HUGR027,0.065174,0.030465,0.054534,0.11257,0.282612,0.347336,0.33158,0.321961,0.246624,0.102236,...,0.56994,0.759909,0.785291,0.771139,0.760384,0.713204,0.600826,0.502033,0.346595,0.561615
HUGR028,0.032908,0.033086,0.064145,0.149769,0.268827,0.294452,0.287196,0.262785,0.202934,0.111363,...,0.588768,0.69014,0.70081,0.689948,0.674858,0.636398,0.578626,0.486094,0.385912,0.539542


In [21]:
# Assign the "basin_id" to the gauges names:
vegetation_df.index.name = "basin_id"

In [22]:
# Assign the "date" to the df index:
LAI_df.index.name = "date"
LAI_yr.index.name = "date"
ndvi_df.index.name = "date"
ndvi_yr.index.name = "date"

In [25]:
# Round the data to 3 decimals
LAI_df = LAI_df.astype(float).round(3)
LAI_yr = LAI_yr.astype(float).round(3)
ndvi_df = ndvi_df.astype(float).round(3)
ndvi_yr = ndvi_yr.astype(float).round(3)
vegetation_df = vegetation_df.astype(float).round(3)

# Data export

In [23]:
# Export the final datasets:
# Time-series:
LAI_df.to_csv(PATH_OUTPUT_TS+"/estreams_LAI_monhtly.csv")
LAI_yr.to_csv(PATH_OUTPUT_TS+"/estreams_LAI_yearly.csv")

ndvi_df.to_csv(PATH_OUTPUT_TS+"/estreams_NDVI_monhtly.csv")
ndvi_yr.to_csv(PATH_OUTPUT_TS+"/estreams_NDVI_yearly.csv")

# Static attributes:
vegetation_df.to_csv(PATH_OUTPUT_ST+"/estreams_vegetation_attributes.csv")

# End